In [1]:
import pandas as pd 
import numpy as np
from tensorflow import keras
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [2]:
df=pd.read_csv('spam.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
def change_cat(val):
    if val=='spam':
        return 1
    return 0

In [4]:
df_spam = df[df['Category']=='spam']
df_spam.shape

(747, 2)

In [5]:
df_ham = df[df['Category']=='ham']
df_ham.shape

(4825, 2)

In [6]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(747, 2)

In [7]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])
df_balanced.shape

(1494, 2)

In [8]:
df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [9]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
4643,spam,You are being ripped off! Get your mobile cont...,1
548,ham,"Sorry that took so long, omw now",0
1069,spam,Someone U know has asked our dating service 2 ...,1
1493,ham,In the end she might still vomit but its okay....,0
4743,ham,Ya very nice. . .be ready on thursday,0


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [11]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [12]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

get_sentence_embeding([
    "500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351695, -0.5132727 , -0.88845736, ..., -0.74748874,
        -0.75314736,  0.91964495],
       [-0.87208354, -0.50543964, -0.94446677, ..., -0.8584749 ,
        -0.7174534 ,  0.88082975]], dtype=float32)>

In [22]:
inputs=keras.Input(shape=(),dtype=tf.string)
bert_vectors=get_sentence_embeding(inputs)
flatten=keras.layers.Flatten()(bert_vectors)
dense=keras.layers.Dense(64,activation='tanh')(flatten)
outputs=keras.layers.Dense(1,activation='sigmoid')(dense)

In [26]:
model=keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [27]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_4[0][0]']                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [28]:
model.fit(X_train,y_train,epochs=3,batch_size=64)

Epoch 1/3
18/18 [==============================] - 151s 8s/step - loss: 0.6630 - accuracy: 0.6205
Epoch 2/3
18/18 [==============================] - 158s 9s/step - loss: 0.3938 - accuracy: 0.8500
Epoch 3/3
18/18 [==============================] - 164s 9s/step - loss: 0.3050 - accuracy: 0.8857


In [30]:
# model.save('spam_using_bert.h5')

In [31]:
model.evaluate(X_test,y_test)

12/12 [==============================] - 54s 4s/step - loss: 0.2833 - accuracy: 0.8904


[0.283313512802124, 0.8903743028640747]

In [36]:
emails = [df.Message[2]]
model.predict(emails)

1/1 [==============================] - 0s 248ms/step


array([[0.8013006]], dtype=float32)